# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [67]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [68]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,22.67,71,19,9.44,MU,1691513100
1,1,guiuan,11.0333,125.7247,29.17,76,84,5.38,PH,1691513100
2,2,bredasdorp,-34.5322,20.0403,10.95,83,55,2.11,ZA,1691513100
3,3,adamstown,-25.0660,-130.1015,22.27,87,98,12.56,PN,1691513100
4,4,avarua,-21.2078,-159.7750,18.03,88,20,1.54,CK,1691513100


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [69]:
# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600    
)

# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [70]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [71]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_new = city_data_df.loc[city_data_df["Max Temp"] < 25]
city_data_new.head()

# Drop any rows with null values
# YOUR CODE HERE
city_data_new.dropna(how='all')
city_data_new

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,22.67,71,19,9.44,MU,1691513100
2,2,bredasdorp,-34.5322,20.0403,10.95,83,55,2.11,ZA,1691513100
3,3,adamstown,-25.0660,-130.1015,22.27,87,98,12.56,PN,1691513100
4,4,avarua,-21.2078,-159.7750,18.03,88,20,1.54,CK,1691513100
6,6,udachny,66.4167,112.4000,19.14,51,45,2.38,RU,1691513100
...,...,...,...,...,...,...,...,...,...,...
559,559,saint-joseph,-21.3667,55.6167,20.21,77,39,3.57,RE,1691513158
560,560,sarupathar,26.1984,93.8094,24.79,99,100,0.49,IN,1691513158
561,561,mount pearl,47.5166,-52.7813,16.85,81,75,7.20,CA,1691513158
562,562,oliver,49.1831,-119.5524,23.14,64,100,3.09,CA,1691513001


### Step 3: Create a new DataFrame called `hotel_df`.

In [72]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_new.copy()
hotel_df

hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
hotel_df.head()
# Display sample data
# YOUR CODE HERE

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,71,
2,bredasdorp,ZA,-34.5322,20.0403,83,
3,adamstown,PN,-25.0660,-130.1015,87,
4,avarua,CK,-21.2078,-159.7750,88,
6,udachny,RU,66.4167,112.4000,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [73]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "format":"json"   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
  
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
bredasdorp - nearest hotel: Victoria Hotel
adamstown - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
udachny - nearest hotel: Вилюй
mahalapye - nearest hotel: Lme Lodge
iqaluit - nearest hotel: Frobisher Inn
kerikeri - nearest hotel: Avalon Resort
pacocha - nearest hotel: Gran Hotel Ilo
jamestown - nearest hotel: DoubleTree Jamestown
waitangi - nearest hotel: Hotel Chathams
margaret river - nearest hotel: Margaret River Hotel
blackmans bay - nearest hotel: Villa Howden
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
hermanus - nearest hotel: Aloe guest house
grytviken - nearest hotel: No hotel found
peace river - nearest hotel: New Ridge Inn
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
dunay - nearest hotel: No hotel found
labytnangi - nearest hotel: Семь лиственниц
punta arenas - nearest hotel: Hotel Lacolet
aleksandrovsk-sakhalinskiy - nearest hotel: Три брата
sitka - neare

qamdo - nearest hotel: 银杏商务宾馆
margate - nearest hotel: The Reading Rooms
kataba - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
igrim - nearest hotel: Факел
tshabong - nearest hotel: No hotel found
thakhek - nearest hotel: 가성비 좋은호텔
karatau - nearest hotel: Артабас
kapuskasing - nearest hotel: Travelodge
badger - nearest hotel: Hotel North Pole
debre tabor - nearest hotel: Kise alemayehu
esik - nearest hotel: No hotel found
kiunga - nearest hotel: No hotel found
villa regina - nearest hotel: Nuevo Cheo
belomorsk - nearest hotel: Гандвик
onega - nearest hotel: Гостевой комплекс "Алёнушка"
namibe - nearest hotel: Hotel Chik Chik Namibe
kuvshinovo - nearest hotel: No hotel found
byron bay - nearest hotel: Lord Byron Resort
galesong - nearest hotel: No hotel found
horta - nearest hotel: Azoris Faial Garden - Resort Hotel
chifeng - nearest hotel: 心连心酒店
raduzhny - nearest hotel: Южная
kapit - nearest hotel: Haip
ballina - nearest hotel: Ballina Beach Res

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,71,Escale Vacances
2,bredasdorp,ZA,-34.5322,20.0403,83,Victoria Hotel
3,adamstown,PN,-25.0660,-130.1015,87,No hotel found
4,avarua,CK,-21.2078,-159.7750,88,Paradise Inn
6,udachny,RU,66.4167,112.4000,51,Вилюй
...,...,...,...,...,...,...
559,saint-joseph,RE,-21.3667,55.6167,77,PALM Hotel & Spa
560,sarupathar,IN,26.1984,93.8094,99,No hotel found
561,mount pearl,CA,47.5166,-52.7813,81,Sandman Signature St. John's
562,oliver,CA,49.1831,-119.5524,64,Coast Oliver Hotel


In [74]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,71,Escale Vacances
2,bredasdorp,ZA,-34.5322,20.0403,83,Victoria Hotel
3,adamstown,PN,-25.0660,-130.1015,87,No hotel found
4,avarua,CK,-21.2078,-159.7750,88,Paradise Inn
6,udachny,RU,66.4167,112.4000,51,Вилюй


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
    
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [75]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)